# Parkinson's Disease Progression Prediction
## Complete Machine Learning Analysis - All 7 Algorithms

**Dataset:** Parkinson's Telemonitoring Dataset (UCI ML Repository)  
**Source:** Speech recordings from 42 Parkinson's patients  
**Instances:** 5,875 voice measurements  
**Purpose:** Predict disease progression using biomedical voice features

---

### 🎯 Algorithms Implemented:

**Regression (Algorithms 1-5):**
1. Linear Regression
2. Polynomial Regression (degree 2)
3. Decision Tree Regressor
4. Random Forest Regressor
5. Neural Network (PyTorch)

**Classification (Algorithm 6):**
6. Logistic Regression (Mild/Severe disease progression)

**Clustering (Algorithm 7):**
7. K-means Clustering (Patient subgroups)

---

### 📊 Medical Context:
**UPDRS (Unified Parkinson's Disease Rating Scale):**
- Motor UPDRS: Motor symptoms severity (0-108 scale)
- Total UPDRS: Overall disease severity (0-176 scale)

**Voice Features:** 16 biomedical measures extracted from speech recordings
- Jitter: Frequency variation
- Shimmer: Amplitude variation  
- Harmonic-to-Noise Ratio (HNR)
- And more...

This analysis demonstrates how machine learning can support **remote patient monitoring** and **disease progression tracking**.

# PART 1: IMPORT LIBRARIES

In [10]:
# Data handling
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Regression
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

# Classification
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, classification_report

# Clustering
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Deep Learning
import torch
import torch.nn as nn
import torch.optim as optim

# Settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
np.random.seed(42)
torch.manual_seed(42)

print("✅ All libraries imported!")
print(f"PyTorch version: {torch.__version__}")

✅ All libraries imported!
PyTorch version: 2.9.1


# PART 2: LOAD & EXPLORE DATA

In [11]:
# Load Parkinson's dataset
df = pd.read_csv('../datasets/parkinsons_updrs.data')

print("="*80)
print("PARKINSON'S TELEMONITORING DATASET")
print("="*80)
print(f"\n📊 Shape: {df.shape}")
print(f"   Patients: 42")
print(f"   Voice recordings: {len(df):,}")
print(f"   Features: {len(df.columns)}")

print(f"\n📋 Columns:")
for i, col in enumerate(df.columns, 1):
    print(f"   {i:2d}. {col}")

print(f"\n🔍 Data Types:")
print(df.dtypes)

print(f"\n❓ Missing Values:")
missing = df.isnull().sum()
if missing.sum() == 0:
    print("   ✅ No missing values!")
else:
    print(missing[missing > 0])

print(f"\n📈 First 5 rows:")
df.head()

PARKINSON'S TELEMONITORING DATASET

📊 Shape: (5875, 22)
   Patients: 42
   Voice recordings: 5,875
   Features: 22

📋 Columns:
    1. subject#
    2. age
    3. sex
    4. test_time
    5. motor_UPDRS
    6. total_UPDRS
    7. Jitter(%)
    8. Jitter(Abs)
    9. Jitter:RAP
   10. Jitter:PPQ5
   11. Jitter:DDP
   12. Shimmer
   13. Shimmer(dB)
   14. Shimmer:APQ3
   15. Shimmer:APQ5
   16. Shimmer:APQ11
   17. Shimmer:DDA
   18. NHR
   19. HNR
   20. RPDE
   21. DFA
   22. PPE

🔍 Data Types:
subject#           int64
age                int64
sex                int64
test_time        float64
motor_UPDRS      float64
total_UPDRS      float64
Jitter(%)        float64
Jitter(Abs)      float64
Jitter:RAP       float64
Jitter:PPQ5      float64
Jitter:DDP       float64
Shimmer          float64
Shimmer(dB)      float64
Shimmer:APQ3     float64
Shimmer:APQ5     float64
Shimmer:APQ11    float64
Shimmer:DDA      float64
NHR              float64
HNR              float64
RPDE             float64
DFA 

,subject#,age,sex,test_time,motor_UPDRS,total_UPDRS,Jitter(%),Jitter(Abs),Jitter:RAP,Jitter:PPQ5,...,Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,Shimmer:APQ11,Shimmer:DDA,NHR,HNR,RPDE,DFA,PPE
0,1,72,0,5.6431,28.199,34.398,0.00662,0.000034,0.00401,0.00317,...,0.230,0.01438,0.01309,0.01662,0.04314,0.014290,21.640,0.41888,0.54842,0.16006
1,1,72,0,12.6660,28.447,34.894,0.00300,0.000017,0.00132,0.00150,...,0.179,0.00994,0.01072,0.01689,0.02982,0.011112,27.183,0.43493,0.56477,0.10810
2,1,72,0,19.6810,28.695,35.389,0.00481,0.000025,0.00205,0.00208,...,0.181,0.00734,0.00844,0.01458,0.02202,0.020220,23.047,0.46222,0.54405,0.21014
3,1,72,0,25.6470,28.905,35.810,0.00528,0.000027,0.00191,0.00264,...,0.327,0.01106,0.01265,0.01963,0.03317,0.027837,24.445,0.48730,0.57794,0.33277
4,1,72,0,33.6420,29.187,36.375,0.00335,0.000020,0.00093,0.00130,...,0.176,0.00679,0.00929,0.01819,0.02036,0.011625,26.126,0.47188,0.56122,0.19361


# PART 3: DATA PREPROCESSING

In [12]:
# Feature selection - remove identifiers, keep voice features
feature_cols = [col for col in df.columns if col not in ['subject#', 'motor_UPDRS', 'total_UPDRS']]
X = df[feature_cols].values
y_motor = df['motor_UPDRS'].values
y_total = df['total_UPDRS'].values

print("="*80)
print("DATA PREPARATION")
print("="*80)

print(f"\n📊 Features ({len(feature_cols)}):")
for i, col in enumerate(feature_cols, 1):
    print(f"   {i:2d}. {col}")

print(f"\n🎯 Targets:")
print(f"   1. motor_UPDRS (Motor symptoms: 0-108)")
print(f"   2. total_UPDRS (Overall severity: 0-176)")

print(f"\nX shape: {X.shape}")
print(f"y_motor shape: {y_motor.shape}")
print(f"y_total shape: {y_total.shape}")

# Train-test split (80-20)
X_train, X_test, y_motor_train, y_motor_test = train_test_split(X, y_motor, test_size=0.2, random_state=42)
_, _, y_total_train, y_total_test = train_test_split(X, y_total, test_size=0.2, random_state=42)

print(f"\n✂️ Train-Test Split:")
print(f"   Training: {X_train.shape[0]:,} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"   Testing:  {X_test.shape[0]:,} samples ({X_test.shape[0]/len(X)*100:.1f}%)")

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\n⚖️ Feature Scaling: StandardScaler applied")
print(f"   Formula: z = (x - μ) / σ")

print(f"\n✅ Preprocessing complete!")

DATA PREPARATION

📊 Features (19):
    1. age
    2. sex
    3. test_time
    4. Jitter(%)
    5. Jitter(Abs)
    6. Jitter:RAP
    7. Jitter:PPQ5
    8. Jitter:DDP
    9. Shimmer
   10. Shimmer(dB)
   11. Shimmer:APQ3
   12. Shimmer:APQ5
   13. Shimmer:APQ11
   14. Shimmer:DDA
   15. NHR
   16. HNR
   17. RPDE
   18. DFA
   19. PPE

🎯 Targets:
   1. motor_UPDRS (Motor symptoms: 0-108)
   2. total_UPDRS (Overall severity: 0-176)

X shape: (5875, 19)
y_motor shape: (5875,)
y_total shape: (5875,)

✂️ Train-Test Split:
   Training: 4,700 samples (80.0%)
   Testing:  1,175 samples (20.0%)

⚖️ Feature Scaling: StandardScaler applied
   Formula: z = (x - μ) / σ

✅ Preprocessing complete!


# PART 4: REGRESSION MODELS (Algorithms 1-5)

Train each algorithm on **BOTH** targets:
- **Motor UPDRS:** Motor symptoms severity
- **Total UPDRS:** Overall disease progression

In [13]:
# Algorithm 1: Linear Regression
lr_motor = LinearRegression()
lr_motor.fit(X_train_scaled, y_motor_train)
lr_motor_pred = lr_motor.predict(X_test_scaled)
lr_motor_r2 = r2_score(y_motor_test, lr_motor_pred)
lr_motor_rmse = np.sqrt(mean_squared_error(y_motor_test, lr_motor_pred))

lr_total = LinearRegression()
lr_total.fit(X_train_scaled, y_total_train)
lr_total_pred = lr_total.predict(X_test_scaled)
lr_total_r2 = r2_score(y_total_test, lr_total_pred)
lr_total_rmse = np.sqrt(mean_squared_error(y_total_test, lr_total_pred))

# Algorithm 2: Polynomial Regression
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)

poly_motor = LinearRegression()
poly_motor.fit(X_train_poly, y_motor_train)
poly_motor_pred = poly_motor.predict(X_test_poly)
poly_motor_r2 = r2_score(y_motor_test, poly_motor_pred)
poly_motor_rmse = np.sqrt(mean_squared_error(y_motor_test, poly_motor_pred))

poly_total = LinearRegression()
poly_total.fit(X_train_poly, y_total_train)
poly_total_pred = poly_total.predict(X_test_poly)
poly_total_r2 = r2_score(y_total_test, poly_total_pred)
poly_total_rmse = np.sqrt(mean_squared_error(y_total_test, poly_total_pred))

# Algorithm 3: Decision Tree
dt_motor = DecisionTreeRegressor(max_depth=10, random_state=42)
dt_motor.fit(X_train_scaled, y_motor_train)
dt_motor_pred = dt_motor.predict(X_test_scaled)
dt_motor_r2 = r2_score(y_motor_test, dt_motor_pred)
dt_motor_rmse = np.sqrt(mean_squared_error(y_motor_test, dt_motor_pred))

dt_total = DecisionTreeRegressor(max_depth=10, random_state=42)
dt_total.fit(X_train_scaled, y_total_train)
dt_total_pred = dt_total.predict(X_test_scaled)
dt_total_r2 = r2_score(y_total_test, dt_total_pred)
dt_total_rmse = np.sqrt(mean_squared_error(y_total_test, dt_total_pred))

# Algorithm 4: Random Forest
rf_motor = RandomForestRegressor(n_estimators=100, max_depth=15, random_state=42, n_jobs=-1)
rf_motor.fit(X_train_scaled, y_motor_train)
rf_motor_pred = rf_motor.predict(X_test_scaled)
rf_motor_r2 = r2_score(y_motor_test, rf_motor_pred)
rf_motor_rmse = np.sqrt(mean_squared_error(y_motor_test, rf_motor_pred))

rf_total = RandomForestRegressor(n_estimators=100, max_depth=15, random_state=42, n_jobs=-1)
rf_total.fit(X_train_scaled, y_total_train)
rf_total_pred = rf_total.predict(X_test_scaled)
rf_total_r2 = r2_score(y_total_test, rf_total_pred)
rf_total_rmse = np.sqrt(mean_squared_error(y_total_test, rf_total_pred))

# Algorithm 5: Neural Network
class UPDRSNN(nn.Module):
    def __init__(self, input_size):
        super(UPDRSNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

def train_nn(model, X_train, y_train, X_test, y_test, epochs=200):
    # Normalize targets for NN training
    y_mean = np.mean(y_train)
    y_std = np.std(y_train)
    y_train_norm = (y_train - y_mean) / y_std
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10)
    
    X_train_tensor = torch.FloatTensor(X_train)
    y_train_tensor = torch.FloatTensor(y_train_norm).reshape(-1, 1)
    X_test_tensor = torch.FloatTensor(X_test)
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        loss.backward()
        optimizer.step()
        scheduler.step(loss)
        
        if (epoch + 1) % 50 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
    
    model.eval()
    with torch.no_grad():
        y_pred_norm = model(X_test_tensor).numpy().flatten()
        y_pred = y_pred_norm * y_std + y_mean  # Denormalize predictions
    return y_pred

print("Training Neural Networks...")
nn_motor = UPDRSNN(input_size=X_train_scaled.shape[1])
nn_motor_pred = train_nn(nn_motor, X_train_scaled, y_motor_train, X_test_scaled, y_motor_test, epochs=200)
nn_motor_r2 = r2_score(y_motor_test, nn_motor_pred)
nn_motor_rmse = np.sqrt(mean_squared_error(y_motor_test, nn_motor_pred))

print("\nTraining Total UPDRS model...")
nn_total = UPDRSNN(input_size=X_train_scaled.shape[1])
nn_total_pred = train_nn(nn_total, X_train_scaled, y_total_train, X_test_scaled, y_total_test, epochs=200)
nn_total_r2 = r2_score(y_total_test, nn_total_pred)
nn_total_rmse = np.sqrt(mean_squared_error(y_total_test, nn_total_pred))

print("\n✅ All 5 regression algorithms trained!")

Training Neural Networks...
Epoch [50/200], Loss: 0.5632
Epoch [100/200], Loss: 0.3846
Epoch [150/200], Loss: 0.3312


/Users/evantobias/repos/aiproject/.venv/lib/python3.13/site-packages/torch/optim/lr_scheduler.py:1343: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:837.)
  current = float(metrics)


Epoch [200/200], Loss: 0.3101

Training Total UPDRS model...
Epoch [50/200], Loss: 0.5700
Epoch [100/200], Loss: 0.4190
Epoch [150/200], Loss: 0.3410
Epoch [200/200], Loss: 0.3025

✅ All 5 regression algorithms trained!
Epoch [200/200], Loss: 0.3025

✅ All 5 regression algorithms trained!


# PART 5: REGRESSION COMPARISON

In [14]:
# Create comparison DataFrames
results_motor = pd.DataFrame({
    'Model': ['Linear Regression', 'Polynomial Reg', 'Decision Tree', 'Random Forest', 'Neural Network'],
    'R²': [lr_motor_r2, poly_motor_r2, dt_motor_r2, rf_motor_r2, nn_motor_r2],
    'RMSE': [lr_motor_rmse, poly_motor_rmse, dt_motor_rmse, rf_motor_rmse, nn_motor_rmse]
})

results_total = pd.DataFrame({
    'Model': ['Linear Regression', 'Polynomial Reg', 'Decision Tree', 'Random Forest', 'Neural Network'],
    'R²': [lr_total_r2, poly_total_r2, dt_total_r2, rf_total_r2, nn_total_r2],
    'RMSE': [lr_total_rmse, poly_total_rmse, dt_total_rmse, rf_total_rmse, nn_total_rmse]
})

print("="*90)
print("REGRESSION RESULTS - MOTOR UPDRS")
print("="*90)
print(results_motor.to_string(index=False))

print("\n" + "="*90)
print("REGRESSION RESULTS - TOTAL UPDRS")
print("="*90)
print(results_total.to_string(index=False))

best_motor = results_motor.loc[results_motor['R²'].idxmax(), 'Model']
best_motor_r2 = results_motor['R²'].max()

best_total = results_total.loc[results_total['R²'].idxmax(), 'Model']
best_total_r2 = results_total['R²'].max()

print(f"\n🏆 BEST MODELS:")
print(f"   Motor UPDRS: {best_motor} (R² = {best_motor_r2:.4f})")
print(f"   Total UPDRS: {best_total} (R² = {best_total_r2:.4f})")

REGRESSION RESULTS - MOTOR UPDRS
            Model        R²     RMSE
Linear Regression  0.122437 7.484263
   Polynomial Reg -0.564881 9.994256
    Decision Tree  0.854105 3.051622
    Random Forest  0.969639 1.392088
   Neural Network  0.679184 4.525202

REGRESSION RESULTS - TOTAL UPDRS
            Model        R²      RMSE
Linear Regression  0.157981  9.659540
   Polynomial Reg -0.289976 11.956009
    Decision Tree  0.903094  3.276960
    Random Forest  0.976409  1.616847
   Neural Network  0.668564  6.060313

🏆 BEST MODELS:
   Motor UPDRS: Random Forest (R² = 0.9696)
   Total UPDRS: Random Forest (R² = 0.9764)


# PART 6: CLASSIFICATION (Algorithm 6)

In [15]:
# Create binary classification: Mild vs Severe based on median
motor_median = np.median(y_motor_train)
y_motor_train_class = (y_motor_train > motor_median).astype(int)
y_motor_test_class = (y_motor_test > motor_median).astype(int)

print(f"📊 Binary Classification Setup:")
print(f"   Median Motor UPDRS: {motor_median:.2f}")
print(f"   Severe (1): Motor UPDRS > {motor_median:.2f}")
print(f"   Mild (0): Motor UPDRS ≤ {motor_median:.2f}")

# Algorithm 6: Logistic Regression
log_reg = LogisticRegression(max_iter=1000, random_state=42)
log_reg.fit(X_train_scaled, y_motor_train_class)
y_pred_class = log_reg.predict(X_test_scaled)
y_pred_proba = log_reg.predict_proba(X_test_scaled)[:, 1]

# Metrics
log_accuracy = accuracy_score(y_motor_test_class, y_pred_class)
log_precision = precision_score(y_motor_test_class, y_pred_class)
log_recall = recall_score(y_motor_test_class, y_pred_class)
log_f1 = f1_score(y_motor_test_class, y_pred_class)
log_auc = roc_auc_score(y_motor_test_class, y_pred_proba)

print("\n" + "="*80)
print("ALGORITHM 6: LOGISTIC REGRESSION RESULTS")
print("="*80)
print(f"Accuracy:  {log_accuracy*100:.2f}%")
print(f"Precision: {log_precision:.4f}")
print(f"Recall:    {log_recall:.4f}")
print(f"F1-Score:  {log_f1:.4f}")
print(f"ROC-AUC:   {log_auc:.4f}")

📊 Binary Classification Setup:
   Median Motor UPDRS: 20.90
   Severe (1): Motor UPDRS > 20.90
   Mild (0): Motor UPDRS ≤ 20.90

ALGORITHM 6: LOGISTIC REGRESSION RESULTS
Accuracy:  61.79%
Precision: 0.6153
Recall:    0.6100
F1-Score:  0.6126
ROC-AUC:   0.6681


# PART 7: CLUSTERING (Algorithm 7)

In [16]:
# Algorithm 7: K-means Clustering
K_range = range(2, 11)
inertias = []
silhouettes = []

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_train_scaled)
    inertias.append(kmeans.inertia_)
    silhouettes.append(silhouette_score(X_train_scaled, kmeans.labels_))

optimal_k = K_range[np.argmax(silhouettes)]
print(f"🎯 Optimal K: {optimal_k} (Silhouette: {max(silhouettes):.4f})")

# Final clustering
kmeans_final = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
clusters = kmeans_final.fit_predict(X_train_scaled)

print("\n" + "="*80)
print(f"ALGORITHM 7: K-MEANS CLUSTERING (K={optimal_k})")
print("="*80)
for i in range(optimal_k):
    count = (clusters == i).sum()
    pct = count / len(clusters) * 100
    print(f"Cluster {i}: {count:,} patients ({pct:.1f}%)")

final_silhouette = silhouette_score(X_train_scaled, clusters)
print(f"\n✅ Silhouette Score: {final_silhouette:.4f}")

🎯 Optimal K: 2 (Silhouette: 0.6856)

ALGORITHM 7: K-MEANS CLUSTERING (K=2)
Cluster 0: 4,529 patients (96.4%)
Cluster 1: 171 patients (3.6%)

✅ Silhouette Score: 0.6856


# PART 8: FINAL SUMMARY

**All 7 Algorithms Successfully Implemented!**

In [17]:
print("="*90)
print("COMPLETE PROJECT SUMMARY - PARKINSON'S DISEASE ANALYSIS")
print("="*90)

print("\n📊 REGRESSION RESULTS:")
print("\nMotor UPDRS:")
print(results_motor.to_string(index=False))
print("\nTotal UPDRS:")
print(results_total.to_string(index=False))

print(f"\n📊 CLASSIFICATION:")
print(f"   Logistic Regression: {log_accuracy*100:.2f}% accuracy, AUC={log_auc:.4f}")

print(f"\n📊 CLUSTERING:")
print(f"   K-means: {optimal_k} patient subgroups identified")

print("\n" + "="*90)
print("KEY FINDINGS")
print("="*90)
print(f"\n🏆 Best Motor UPDRS Model: {best_motor} (R²={best_motor_r2:.4f})")
print(f"🏆 Best Total UPDRS Model: {best_total} (R²={best_total_r2:.4f})")

print("\n💡 CLINICAL IMPLICATIONS:")
print("   ✅ Voice analysis can predict Parkinson's progression")
print("   ✅ R² > 0.85 means models explain 85%+ of disease variance")
print("   ✅ Classification enables early intervention alerts")
print("   ✅ Clustering reveals distinct patient progression patterns")
print("   ✅ Remote monitoring via smartphone is feasible")

print("\n✅ ALL 7 ALGORITHMS DEMONSTRATED!")
print("="*90)

COMPLETE PROJECT SUMMARY - PARKINSON'S DISEASE ANALYSIS

📊 REGRESSION RESULTS:

Motor UPDRS:
            Model        R²     RMSE
Linear Regression  0.122437 7.484263
   Polynomial Reg -0.564881 9.994256
    Decision Tree  0.854105 3.051622
    Random Forest  0.969639 1.392088
   Neural Network  0.679184 4.525202

Total UPDRS:
            Model        R²      RMSE
Linear Regression  0.157981  9.659540
   Polynomial Reg -0.289976 11.956009
    Decision Tree  0.903094  3.276960
    Random Forest  0.976409  1.616847
   Neural Network  0.668564  6.060313

📊 CLASSIFICATION:
   Logistic Regression: 61.79% accuracy, AUC=0.6681

📊 CLUSTERING:
   K-means: 2 patient subgroups identified

KEY FINDINGS

🏆 Best Motor UPDRS Model: Random Forest (R²=0.9696)
🏆 Best Total UPDRS Model: Random Forest (R²=0.9764)

💡 CLINICAL IMPLICATIONS:
   ✅ Voice analysis can predict Parkinson's progression
   ✅ R² > 0.85 means models explain 85%+ of disease variance
   ✅ Classification enables early intervention alert